<h1> Investigation of Data Aggregation in Pandas

In [11]:
# Load the data into the pandas dataframe
import pandas as pd


df = pd.read_csv("data_sp18.csv") # Modify to correct data location
# print(df.head())
print(df.columns)
# help(df)

Index(['Term Code', 'College Code', 'Department Code', 'CRN', 'Campus Code',
       'Subject Code', 'Course Number', 'Section Number', 'Section Title',
       'Instructor 1 ID', 'Instructor 1 First Name', 'Instructor 1 Last Name',
       'Question Number', 'Question', 'Mean', 'Median', 'Standard Deviation',
       'Department Mean', 'Department Median', 'Similar College Mean',
       'Similar College Median', 'College Mean', 'College Median',
       'Percent Rank - Department', 'Percent Rank - College', 'Responses',
       'Percent #1', 'Percent #2', 'Percent #3', 'Percent #4', 'Percent #5',
       'ZScore - College', 'ZScore - College Similar Sections', 'Course Level',
       'Section Size'],
      dtype='object')


In [278]:
import numpy as np
import pandas as pd

df = pd.read_csv("data/data_sp18.csv") # Modify to correct data location

# Initialize the aggregated dataframe by copying the base data frame
ag_df = df.copy()

# Drop the unnecessary columns
ag_df.drop(['Department Code', 'Question Number','Section Number','Responses', 'CRN','Campus Code','Question', 'Mean', 'Median', 'Standard Deviation', 'Department Mean', 'Department Median', 'Similar College Mean', 'College Mean', 'College Median', 'Percent Rank - Department', 'Percent Rank - College', 'Percent #1', 'Percent #2', 'Percent #3', 'Percent #4', 'Percent #5', 'ZScore - College', 'ZScore - College Similar Sections', 'Course Level', 'Section Size', 'Similar College Median'], axis=1, inplace = True)


# Add in the columns to be filled with the aggregated values
ag_df.insert(3,'Avg Department Rating', 0.0)
ag_df.insert(4,'SD Department Rating', 0.0)
ag_df.insert(7,'Avg Course Rating', 0.0)
ag_df.insert(8,'SD Course Rating', 0.0)
ag_df.insert(12, 'Avg Instructor Rating In Section', 0.0)
ag_df.insert(13, 'SD Instructor Rating In Section', 0.0)
ag_df.insert(14, 'Num Responses', 0)
# # Swap the columns Section Number and Section Title
# swap_col_order = list(ag_df.columns)
# # swap_col_order[6] = 'Instructor ID'
# # swap_col_order[9] = 'Section Number'
# ag_df = ag_df.reindex(columns = swap_col_order)

# Rename the Instructor 1 ID, Instructor 1 First Name, Last Name columns
ag_df.rename(columns = {'Section Title':'Course Title', 'Responses':'Num Responses','Instructor 1 ID': 'Instructor ID', 'Instructor 1 First Name':'Instructor First Name', 'Instructor 1 Last Name':'Instructor Last Name'}, inplace= True)

# Remove the repeat rows that will occur because we are taking 1-10 question responses down to 1
ag_df.drop_duplicates(subset = ag_df.columns.drop('Course Title'), inplace = True)

# Now we have the column headings we want
# Fill the columns with correct values
# print(ag_df.head())
# ag_df[(ag_df['Subject Code']=='DSA') & (ag_df['Course Number']==4413)]
import sys
# Read in the question mappings values from the mappings.yaml
import yaml
with open('mappings.yaml') as f:
    # use safe_load instead load
    mappings = yaml.safe_load(f)
    question_weighting = mappings['Instructor_question_weighting']

# Lets fill the average instructor ranking in each section, i.e. the combined rating for each question per section
for subject in df['Subject Code'].unique(): # Iterate over all subjects (test case - for subject in ['DSA']:)
    for course in df[(df['Subject Code']==subject)]['Course Number'].unique(): # Iterate over courses with the desired subject 
        for instructor in df[(df['Subject Code']==subject) & (df['Course Number']==course)]['Instructor 1 ID'].unique(): # Iterate over instructors with desired subject and course number
            # Modify the subset based on the desired instructor (see section index above)
            subset = df[(df['Subject Code']==subject) & (df['Course Number']==course) & (df['Instructor 1 ID']==instructor)]        
            if len(subset)!=0: 
                # Compute the combined mean and standard deviation of the questions
                # Input the standard deviation, mean, number of responses, and the question number mapped to the weights for each subject-course-instructor combination
#                 print(len(subset['Standard Deviation']))
                instructor_mean, instructor_sd = combine_standard_deviations(subset['Standard Deviation'], subset['Mean'], subset['Responses'], subset['Question Number'].map(arg=question_weighting))
                
                # Set the combined mean and combined sd value into the aggregated dataframe
                # Find the row of interest in the aggregated df
                ag_df_section_row = ag_df[(ag_df['Subject Code']==subject) & (ag_df['Course Number']==course) & (ag_df['Instructor ID']==instructor)].index.tolist()
                if len(ag_df_section_row)!=1:
                    print('Aggregated Dataframe contains incorrect number of entries (number entries =' + str(len(ag_df_section_row))+ ') for subject: '+ str(subject)+ ', course: '+ str(course)+ ', and instructor: '+ str(instructor))
                else:
                    # Fill the Instructor Ratings Columns
                    ag_df.at[ag_df_section_row[0], 'Avg Instructor Rating In Section'] = instructor_mean
                    ag_df.at[ag_df_section_row[0], 'SD Instructor Rating In Section'] = instructor_sd
                    
                    # Fill the Num Responses column, based on the minimum number of responses of the group of questions
                    ag_df.at[ag_df_section_row[0], 'Num Responses'] = min(list(subset['Responses']))
                    
        # Back to Course level of tree, now that we've filled out the instructor level info
        # Modify the dataframe subset that consists only of the entries with the desired course (see course index above)
        # Note that now our subset consists of aggregated data from all instructors within the desired course
        subset = ag_df[(ag_df['Subject Code']==subject) & (ag_df['Course Number']==course)]
        # Compute the combined mean and standard deviation of all of the instructors within the course
        #### IMPORTANT #### NO POPULATION-BASED OR OTHER WEIGHTING USED IN THE CALCULATION OF SD AND AVERAGE COURSE RATING
        
        course_mean, course_sd = combine_standard_deviations(subset['SD Instructor Rating In Section'], subset['Avg Instructor Rating In Section'], np.ones(len(subset['SD Instructor Rating In Section'])), np.ones(len(subset['SD Instructor Rating In Section'])))
        # Find the row of interest in the desired df
        ag_df_course_rows = ag_df[(ag_df['Subject Code']==subject) & (ag_df['Course Number']==course)].index.tolist()
        # Fill the Course ratings columns
        ag_df.at[ag_df_course_rows, 'Avg Course Rating'] = course_mean
        ag_df.at[ag_df_course_rows, 'SD Course Rating'] = course_sd
        
    # Back to Department level of tree, now that we've filled out the instructor and course level info
    # Modify the dataframe subset that consists only of the entries with the desired subject(see course index above)
    # Note that now our subset consists of aggregated data from all instructors and courses within the desired subject/department
    subset = ag_df[(ag_df['Subject Code']==subject)]
    # Compute the combined mean and standard deviation of all of the courses within the department
    #### IMPORTANT #### NO POPULATION-BASED OR OTHER WEIGHTING USED IN THE CALCULATION OF SD AND AVERAGE COURSE RATING

    department_mean, department_sd = combine_standard_deviations(subset['SD Course Rating'], subset['Avg Course Rating'], np.ones(len(subset['Avg Course Rating'])), np.ones(len(subset['Avg Course Rating'])))
    # Find the row of interest in the desired df
    ag_df_course_rows = ag_df[(ag_df['Subject Code']==subject)].index.tolist()
    # Fill the Course ratings columns
    ag_df.at[ag_df_course_rows, 'Avg Department Rating'] = department_mean
    ag_df.at[ag_df_course_rows, 'SD Department Rating'] = department_sd
        
        
# subject = 'DSA'
# course = 4413
# section = 995
# print(ag_df.columns)
# Create the subset based on the desired subject, course, and section
# subset = (df[(df['Subject Code']==subject) & (df['Course Number']==course) & (df['Section Number']==section)])

# Create 1D lists based on the desired column
# ss_mean = subset['Mean'])
# ss_sd = list(subset['Standard Deviation'])
# ss_qnums = subset['Question Number']
# ss_pops = list(subset['Responses'])
# ss_weights = list(ss_qnums.map(arg=question_weighting))
# # Find the appropriate lists to pass to 
# ag_df.head(20)


10
10
10
15
5
5
10
5
5
5
10
5
5
5
10
5
5
5
5
10
10
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
10
5
10
5
5
5
20
20
35
5
5
5
10
5
15
5
5
5
5
5
5
5
5
5
5
15
5
5
5
5
5
5
25
5
5
5
5
5
5
5
5
5
5
5
5
5
10
10
10
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
25
5
5
15
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
15
10
25
10
10
5
5
5
10
10
20
5
5
5
5
5
5
5
5
5
5
5
5
5
5
15
45
5
5
5
5
5
5
5
30
5
5
20
5
5
5
5
5
5
5
20
5
5
5
5
5
5
5
5
5
5
5
5
10
5
5
5
5
5
5
5
5
5
5
15
5
5
5
5
10
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [283]:
# print(ag_df.head(20)['Instructor Last Name'])
df.head(60)[['Instructor 1 Last Name','Responses']]

Instructor 1 Last Name  Responses
0          Lakshmivarahan         10
1          Lakshmivarahan         10
2          Lakshmivarahan         10
3          Lakshmivarahan         10
4          Lakshmivarahan         10
5          Lakshmivarahan          5
6          Lakshmivarahan          5
7          Lakshmivarahan          5
8          Lakshmivarahan          5
9          Lakshmivarahan          5
10                   Yoon          4
11                   Yoon          4
12                   Yoon          4
13                   Yoon          4
14                   Yoon          4
15                   Yoon          4
16                   Yoon          4
17                   Yoon          3
18                   Yoon          4
19                   Yoon          4
20                Stewart         13
21                Stewart         13
22                Stewart         12
23                Stewart         13
24                Stewart         13
25                Stewart          3
26                Stewart          3
27                Stewart          3
28                Stewart          3
29                Stewart          3
30              Nicholson         25
31              Nicholson         25
32              Nicholson         25
33              Nicholson         25
34              Nicholson         25
35              Nicholson         13
36              Nicholson         13
37              Nicholson         13
38              Nicholson         13
39              Nicholson         13
40              Nicholson          3
41              Nicholson          3
42              Nicholson          3
43              Nicholson          3
44              Nicholson          3
45         Lakshmivarahan          8
46         Lakshmivarahan          8
47         Lakshmivarahan          8
48         Lakshmivarahan          8
49         Lakshmivarahan          8
50                 Shehab         11
51                 Shehab         11
52                 Shehab         11
53                 Shehab         11
54                 Shehab         11
55                Stewart          7
56                Stewart          7
57                Stewart          7
58                Stewart          7
59                Stewart          7

In [268]:
ag_df.columns.drop('Course Title')
ag_df.columns


Index(['Term Code', 'College Code', 'Subject Code', 'Avg Department Rating',
       'SD Department Rating', 'Course Number', 'Course Title',
       'Avg Course Rating', 'SD Course Rating', 'Instructor ID',
       'Instructor First Name', 'Instructor Last Name',
       'Avg Instructor Rating In Section', 'SD Instructor Rating In Section',
       'Num Responses'],
      dtype='object')

The formula that we will use for computing and combining the ratings from the available question data is shown, without weighting, below. The image is available from [here](https://www.researchgate.net/post/How_to_combine_standard_deviations_for_three_groups)

![sd_image](https://www.researchgate.net/profile/Twan_Ten_Haaf/post/How_to_combine_standard_deviations_for_three_groups/attachment/59d64a7479197b80779a4bdd/AS%3A475377563901954%401490350251587/download/Grouped_SD.jpg)

In the above formula, the i index corresponds to the question number. In addition to the $ n_i$ and $(n-1)_i$ terms, our formula uses a w_i term for the weighting of each question; That is, each question response and standard deviation is weighted *not only* by the number of responses but also by the weight for that specific question. These formulae will be used throughout the data aggregation for combining data for each section and also for combining standard deviations for each course and each department. Also, the implemented formula does not use the $n-1$, but instead just uses n (minor error for n>5)

In [192]:
def combine_means(mean_list, pop_list, weight_list):
    '''
    This function takes lists of means, population sizes, and weights for each population, and combines the result into a single mean value.
    * All lists must be the same length
    '''
    import numpy as np
    mean_list = np.array(mean_list)
    pop_list = np.array(pop_list)
    weight_list = np.array(weight_list)
    # Check for equal sized lists
    if not len(mean_list)==len(pop_list)==len(weight_list):
        print('All input lists to the function -- combine_standard_deviations -- must be of the same length.')
        # Proceed with the program
    # Combine the population-and-weight-modulated means
    mean = np.sum(mean_list*pop_list*weight_list)/(np.sum(pop_list*weight_list))
    return mean
    

def combine_standard_deviations(sd_list, mean_list,pop_list, weight_list):
    '''
    This function will take lists of standard deviations, means, population sizes, and weights for each list unit. The function
    will combine the lists to produce a standard deviation for the group, based on the input parameters. Formula for combining the SD taken from the below link:
    
    https://www.researchgate.net/post/How_to_combine_standard_deviations_for_three_groups
    
    * All lists must be same length
    '''
    import numpy as np
    # Convert all input lists into numpy arrays
    sd_list = np.array(sd_list)
    mean_list = np.array(mean_list)
    pop_list = np.array(pop_list)
    weight_list = np.array(weight_list)
    # Check for equal sized lists
    if not len(sd_list) == len(mean_list)==len(pop_list)==len(weight_list):
        print('All input lists to the function -- combine_standard_deviations -- must be of the same length.')
    # Proceed with the program
    # Compute the weighted mean of the populations
    pop_mean = combine_means(mean_list, pop_list, weight_list)
    # Compute the deviance
    deviance = np.sum((pop_list)*(weight_list)*(sd_list**2) + (pop_list*weight_list)*(mean_list - pop_mean)**2)
    # Compute the standard deviation
    sd = np.sqrt(deviance/(np.sum(pop_list*weight_list)))
    return pop_mean,sd



In [143]:
print(combine_means([2,9], [10,10], [0.5,1]))
combine_standard_deviations([4,6],[50,9], [47,100], [1,1])

# Validated the above functions for combined sd and combined means with this website, albeit there was No entry or validation for the weighting
# https://www.statstodo.com/CombineMeansSDs_Pgm.php

6.666666666666667
pop mean is 22.108843537414966


19.880297993804273

In [8]:
import yaml
with open('mappings.yaml') as f:
    # use safe_load instead load
    mappings = yaml.safe_load(f)
Term_code_dict = mappings['Term_code_dict']
College_code_dict = mappings['College_code_dict']
Department_code_dict = mappings['Department_code_dict']
Department_code_dict


{'EN00': 'ENGR', 'EN01': 'AME', 'EN02': 'CH E', 'EN03': 'CEES'}

In [94]:
import numpy as np
a = np.array(np.array(np.array([1,2,3])))

a**2


array([1, 4, 9])